<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/%E7%AE%A1%E7%90%86%E6%97%A5%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://blog.csdn.net/AAI666666/article/details/138466713

In [25]:
import sqlite3

# 创建一个新的SQLite数据库，如果文件不存在将会被创建
conn = sqlite3.connect('./langchain.db')

# 创建一个Cursor对象并使用它来执行SQL命令
c = conn.cursor()

# 创建一个表
c.execute('''
create table if not exists schedules
(
    id  INTEGER  primary key autoincrement,
    start_time  TEXT default (strftime('%Y-%m-%d %H:%M:%S', 'now', 'localtime')) not null,
    description text default ''
);
''')

# 提交更改并关闭连接
conn.commit()
conn.close()
print("数据库和表已成功创建！")


数据库和表已成功创建！


In [ ]:
!pip install --upgrade --quiet  pillow
!pip install langchain langchain_community
!pip install --upgrade httpx httpx-sse PyJWT

In [63]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

from langchain_community.chat_models import ChatZhipuAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)


In [64]:
def get_by_date(query_date : str) -> str:
    # 计算时间的年月日
    if "明天" in query_date:
        target_day = datetime.now() + timedelta(days=1)
    elif "后天" in query_date:
        target_day = datetime.now() + timedelta(days=2)
    elif "前天" in query_date:
        target_day = datetime.now() - timedelta(days=2)
    elif "昨天" in query_date:
        target_day = datetime.now() - timedelta(days=1)
    else:
        return None;

    # 打印年月日
    print(f"年: {target_day.year}, 月: {target_day.month}, 日: {target_day.day}")
    return f"{target_day.year}-{target_day.month}-{target_day.day}"

In [65]:
from datetime import datetime, timedelta

def connect_db():
    """ 连接到数据库 """
    conn = sqlite3.connect('langchain.db')
    return conn

def getDatetime():
  return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def add_schedule(description : str) -> str:
    """ 新增日程，比如2024-05-03 20:00:00, 周会 """
    start_time = get_by_date(description)
    print(start_time,description)
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO schedules (start_time, description) VALUES (?, ?);
    """, (start_time, description,))
    conn.commit()
    conn.close()
    return "true"


def delete_schedule_by_time(description : str) -> str:
    """ 根据时间删除日程 """
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        DELETE FROM schedules WHERE start_time = ?;
    """, (start_time,))
    conn.commit()
    conn.close()
    return "true"


def get_schedules_by_date(query_date : str) -> str:
    """ 根据日期查询日程，比如 获取2024-05-03的所有日程 """
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT start_time, description FROM schedules WHERE start_time LIKE ?;
    """, (f"{query_date}%",))
    schedules = cursor.fetchall()
    conn.close()
    return str(schedules)



In [66]:
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
tools = [
    Tool(
        name="add_schedule",
        func=add_schedule,
        description="Add a new schedule to the database."
    ),
    Tool(
        name="delete_schedule_by_time",
        func=delete_schedule_by_time,
        description="Delete a schedule by its start time."
    ),
    Tool(
        name="get_schedules_by_date",
        func=get_schedules_by_date,
        description="Get all schedules for a specific date."
    )

]


# 绑定工具到语言模型
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory=ConversationBufferMemory(),
    agent="zero-shot-react-description"
)

# 使用代理
#agent.run("明天是什么时间?")


In [67]:
question = "创建一套明天晚上6点的日程，开周会"
# start_time = get_by_date(question)
# print(start_time)

# 调用agent.run，并提供参数
result = agent.run(question)
result

年: 2024, 月: 6, 日: 22
2024-6-22 description='明天晚上6点，开周会'
Observation:


OperationalError: database is locked